Все ответы принимаются только с выведением ID каждого из клиентов, подходящих под условия задачи/ В задачах со средним выводить ID не нужно

Какой совокупный объем средств в среднем у клиентов, которые приобрели ипотеку и у клиентов, которые её не приобретали?
Есть ли разница в среднем по каждому из типов баланса у клиентов, которые приобрели ипотеку и у клиентов, которые её не приобретали?
Выберите самого возрастного получателя ипотеки.
Выберите клиента, который раньше всех получил ипотеку.
Создайте любую базу данных в sqlite3, состоящую из 5 колонок, одна из колонок обязательно должна быть типа REAL. (Названия колонок любые, какие захотите)
Добавьте 3 полностью заполненные строки в вашу БД.
Удалите строку из колонки типа REAL (из пункта 8), в которой значение признака среднее (то есть не минимальное и не максимальное). Напрямую указывать значение из этой строки нельзя.

In [11]:
import pandas as pd
from pandasql import sqldf

In [14]:
pysqldf = lambda q: sqldf(q, globals())

In [ ]:
!pip install openpyxl

In [ ]:
!pip install pandasql

In [ ]:
Опишите имеющиеся колонки в данных - сделайте аналог describe в pandas для колонок.

In [18]:
#Посчитайте совокупный объем средств у клиентов в банке - сумма по всем балансам.
bank_data = pd.read_excel("C:\\Users\\Марина\\1. МФТИ\\Аналитика\\bank_account_data.xlsx")
pysqldf("""SELECT 
        Cust_ID,
        Cheq_Bal+Sav_Bal+Debt_Bal AS TOTAL_bal
        FROM bank_data""")



,Cust_ID,TOTAL_bal
0,1,41221.59
1,2,41413.90
2,3,33055.88
3,4,42272.94
4,5,27464.44
...,...,...
195,196,33573.27
196,197,32763.17
197,198,36788.96
198,199,26379.16


In [ ]:
#Выведите только тех клиентов, которые приобрели ипотеку (колонка mortgage).
bank_data = pd.read_excel("C:\\Users\\Марина\\1. МФТИ\\Аналитика\\bank_account_data.xlsx")
pysqldf("""SELECT * 
        FROM bank_data
        WHERE Purchase = 1        
        """)



In [ ]:
#Какой совокупный объем средств в среднем у клиентов, которые приобрели ипотеку и у клиентов, которые её не приобретали?

bank_data = pd.read_excel("C:\\Users\\Марина\\1. МФТИ\\Аналитика\\bank_account_data.xlsx")
pysqldf("""SELECT * 
        FROM bank_data
        WHERE Purchase = 1        
        """)
